In [29]:
# Reading the data
import pandas as pd

df = pd.read_csv('temiz_turkce_haber_4k.csv')
df.head()

,Unnamed: 0,ctext,text
0,0,"Iğdır'da bir kişi, tartıştığı karısını silahla...","Iğdır'da, Umut adlı erkek evi terk eden 1 çocu..."
1,1,Cumhurbaşkanı Recep Tayyip Erdoğan'ın katılımı...,Karsan Otomotiv'den yapılan açıklamada iş insa...
2,2,"Türkiye'nin Otomobili'nin, Bilişim Vadisi'ndek...","Türkiye'nin Otomobili'nin, Bilişim Vadisi'nde ..."
3,3,Van'ın Erciş ilçesinde evden çıktıktan sonra b...,Van'ın Erciş ilçesinde evden çıktıktan sonra b...
4,4,Son dönemin parlayan yıldızlarından olan başar...,"Dünyaca ünlü şarkıcı Rita Ora, annesi Vera Ora..."


In [30]:
df['ctext'][0], df['text'][0]

("Iğdır'da bir kişi, tartıştığı karısını silahla vurarak öldürdükten sonra intihar girişiminde bulundu. EVİ TERK EDEN EŞİNİ IĞDIR'DA BULDUOlay, saat 18.00 sıralarında Kazım Karabekir Caddesi'nde meydana geldi. Ağrı'nın Doğubayazıt ilçesinde oturan 1 çocuk annesi Ebru A., yaklaşık 3 ay önce henüz bilinmeyen nedenle eşini terk edip, Iğdır'a geldi. Burada ev kiralayan Ebru A.'yı bulan eşi Umut A. kente geldi. Eşinin evine giden Umut A., kapıyı çalıp, içeri girdi. ARALARINDA TARTIŞMA ÇIKTI, EŞİNE ATEŞ ETTİBu sırada iddiaya göre, ikili arasında tartışma çıktı. Tartışmanın büyümesi üzerine Umut A., yanında getirdiği tabanca ile Ebru A.'ya ateş etti. Ebru A. kanlar içinde yere yığılırken, Umut A. kendisini de göğsünden vurdu. KADIN EVDE ÖLDÜ, ŞÜPHELİ HASTANEDE TEDAVİ GÖRÜYORSilah sesini duyan komşuların ihbarı üzerine olay yerine polis ve sağlık ekipleri sevk edildi. Polis, Ebru A.'yı banyoda, Umut A.'yı ise odada kanlar içinde buldu. Sağlık görevlilerince yapılan kontrolde Ebru A.'nın yaşamı

In [31]:
#Remove the null values
df=df.dropna()

In [32]:
#Reset the index numbers
df=df.reset_index()

In [33]:
# Textrank function
import networkx as nx
import numpy as np
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

def textrank(document):

    sentence_tokenizer = PunktSentenceTokenizer()
    sentences = sentence_tokenizer.tokenize(document)
 
    bow_matrix = CountVectorizer().fit_transform(sentences)
    normalized = TfidfTransformer().fit_transform(bow_matrix)
 
    similarity_graph = normalized * normalized.T
 
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    scores = nx.pagerank(nx_graph)
    ranked = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    temp = dict(ranked)
    select_rank = [a_tuple[0] for a_tuple in ranked] 
    
    '''
    if len(ranked) == 1:
      res = [temp.get(t, 0) for t in select_rank[:1]]
      summary = ' '.join([str(elem) for elem in res)
    
    elif len(ranked) <= 5:
      res = [temp.get(t, 0) for t in select_rank[:int(len(select_rank)/2)]]
      summary = ' '.join([str(elem) for elem in res])
    
    else:
      '''

    # Setting to comparison of two sentences in bleu score
    res = [temp.get(t, 0) for t in select_rank[:1]]
    summary = ' '.join([str(elem) for elem in res])
       
    return summary

In [34]:
#Append first 1000 text in Turkish News data 
t=[]
for i in range(0,1000):
  txt = df['text'][i]
  results = textrank(txt)
  t.append(results)

In [36]:
#Creating DataFrame for Comparison of Bleu Score
raw = pd.DataFrame()

raw['Original Summary'] = df['text'][:1000]
raw['Textrank Summary'] = t

raw.head()

,Original Summary,Textrank Summary
0,"Iğdır'da, Umut adlı erkek evi terk eden 1 çocu...","Iğdır'da, Umut adlı erkek evi terk eden 1 çocu..."
1,Karsan Otomotiv'den yapılan açıklamada iş insa...,Karsan Otomotiv'den yapılan açıklamada iş insa...
2,"Türkiye'nin Otomobili'nin, Bilişim Vadisi'nde ...","Türkiye'nin Otomobili'nin, Bilişim Vadisi'nde ..."
3,Van'ın Erciş ilçesinde evden çıktıktan sonra b...,Van'ın Erciş ilçesinde evden çıktıktan sonra b...
4,"Dünyaca ünlü şarkıcı Rita Ora, annesi Vera Ora...","Dünyaca ünlü şarkıcı Rita Ora, annesi Vera Ora..."


In [37]:
raw['Original Summary'][0], raw['Textrank Summary'][0]

("Iğdır'da, Umut adlı erkek evi terk eden 1 çocuk annesi eşini tabancayla vurarak öldürdü, ardından kendisini göğsünden vurdu. Yaralanan süpheli, hastanede tedaviye alındı.",
 "Iğdır'da, Umut adlı erkek evi terk eden 1 çocuk annesi eşini tabancayla vurarak öldürdü, ardından kendisini göğsünden vurdu.")

In [38]:
!pip install rouge

from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

rouge = Rouge()

In [39]:
l=[]
for s in range(0,len(raw)):
  print(raw['Original Summary'][s])
  print(raw['Textrank Summary'][s])
  print('BLEU score -> {}'.format(sentence_bleu([raw['Original Summary'][s].split(' ')], raw['Textrank Summary'][s].split(' '))))
  print(rouge.get_scores(raw['Textrank Summary'][s], raw['Original Summary'][s]))
  l.append(sentence_bleu([raw['Original Summary'][s].split(' ')], raw['Textrank Summary'][s].split(' ')))

Iğdır'da, Umut adlı erkek evi terk eden 1 çocuk annesi eşini tabancayla vurarak öldürdü, ardından kendisini göğsünden vurdu. Yaralanan süpheli, hastanede tedaviye alındı.
Iğdır'da, Umut adlı erkek evi terk eden 1 çocuk annesi eşini tabancayla vurarak öldürdü, ardından kendisini göğsünden vurdu.
BLEU score -> 0.7574651283969664
[{'rouge-1': {'f': 0.8780487755621655, 'p': 1.0, 'r': 0.782608695652174}, 'rouge-2': {'f': 0.8717948668770547, 'p': 1.0, 'r': 0.7727272727272727}, 'rouge-l': {'f': 0.8780487755621655, 'p': 1.0, 'r': 0.782608695652174}}]
Karsan Otomotiv'den yapılan açıklamada iş insanı Kıraç'ın sahibi olduğu Kök Ulaşım Taşımacılık A.Ş.'nin, Türkiye'nin Otomobili Girişim Grubu'nun yüzde 19'luk hissesine sahip olduğu belirtildi.
Karsan Otomotiv'den yapılan açıklamada iş insanı Kıraç'ın sahibi olduğu Kök Ulaşım Taşımacılık A.Ş.
BLEU score -> 0.4226706686588658
[{'rouge-1': {'f': 0.7179487133464827, 'p': 1.0, 'r': 0.56}, 'rouge-2': {'f': 0.7027026981446312, 'p': 1.0, 'r': 0.5416666666

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Eski Başbakan Bülent Ecevit'in eşi, Demokratik Sol Parti'nin kurucu ismi Rahşan Ecevit'in hayatını kaybetmesinin ardından birçok ünlü isim duygu dolu taziye mesajları paylaştı.
BLEU score -> 1.0
[{'rouge-1': {'f': 0.999999995, 'p': 1.0, 'r': 1.0}, 'rouge-2': {'f': 0.999999995, 'p': 1.0, 'r': 1.0}, 'rouge-l': {'f': 0.999999995, 'p': 1.0, 'r': 1.0}}]
Kraliyet ailesindeki üst düzey görevlerinden ayrılarak ekonomik bağımsızlıkları için çalışma kararı alan Sussex çiftinden Meghan Markle'a, ünlü yetişkin film sitesinden iş teklifi geldi.
Kraliyet ailesindeki üst düzey görevlerinden ayrılarak ekonomik bağımsızlıkları için çalışma kararı alan Sussex çiftinden Meghan Markle'a, ünlü yetişkin film sitesinden iş teklifi geldi.
BLEU score -> 1.0
[{'rouge-1': {'f': 0.999999995, 'p': 1.0, 'r': 1.0}, 'rouge-2': {'f': 0.999999995, 'p': 1.0, 'r': 1.0}, 'rouge-l': {'f': 0.999999995, 'p': 1.0, 'r': 1.0}}]
Mersin'de ormanlık bir alanda, Ağustos ayında av tüfeği ile vurulmuş halde ölü bulunan Kemal Büyükden

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



[{'rouge-1': {'f': 0.46666666308888893, 'p': 1.0, 'r': 0.30434782608695654}, 'rouge-2': {'f': 0.42857142520408165, 'p': 1.0, 'r': 0.2727272727272727}, 'rouge-l': {'f': 0.48275861702734846, 'p': 1.0, 'r': 0.3181818181818182}}]
Dallas Mavericks forması giyen Luka Doncic, bir karşılaşmada 20 sayı, 15 ribaunt, 15 asist barajlarını aşan en genç oyuncu olarak NBA tarihine geçti.
Dallas Mavericks forması giyen Luka Doncic, bir karşılaşmada 20 sayı, 15 ribaunt, 15 asist barajlarını aşan en genç oyuncu olarak NBA tarihine geçti.
BLEU score -> 1.0
[{'rouge-1': {'f': 0.999999995, 'p': 1.0, 'r': 1.0}, 'rouge-2': {'f': 0.999999995, 'p': 1.0, 'r': 1.0}, 'rouge-l': {'f': 0.999999995, 'p': 1.0, 'r': 1.0}}]
Almanya'da yaşayan ve eşi için ölüm büyüsü yaptırmak isteyen bir kadın, insanlara metafizik yollarla yardım ettiğini belirten kişi tarafından 8 bin euro dolandırıldı.
Almanya'da yaşayan ve eşi için ölüm büyüsü yaptırmak isteyen bir kadın, insanlara metafizik yollarla yardım ettiğini belirten kişi t

In [40]:
# Average of the Textrank Bleu Score
import numpy as np

np.mean(l)

0.7135593055687091

In [41]:
# Average of the Textrank Rouge Score
rouge.get_scores(raw['Textrank Summary'], raw['Original Summary'],avg=True)

{'rouge-1': {'f': 0.8371161844905589, 'p': 1.0, 'r': 0.7701010869565212},
 'rouge-2': {'f': 0.8262685465465854, 'p': 0.995, 'r': 0.7596864295125163},
 'rouge-l': {'f': 0.8417572600886529, 'p': 1.0, 'r': 0.7752823657463813}}